In [34]:
import pandas as pd

LOAD DATASET

In [35]:
df = pd.read_csv("../../data/cleaned.csv")
print("\nRAW DATA PREVIEW:")
print(df.head())


RAW DATA PREVIEW:
         Film_Name Release_Date     Category Language  Viewer_Rate  \
0  Chennai Express   2021-09-12      Romance    Hindi          4.5   
1   Mountain Trail   2020-04-27       Comedy  English          3.5   
2     Eternal Hope   2020-07-04      Romance  English          4.6   
3  Seoul Heartbeat   2020-04-17       Comedy  English          4.1   
4      Shadow Pact   2022-02-24  Documentary  English          4.3   

   Number_of_Views Viewing_Month  Release_Year  Release_Month  Viewing_Year  \
0            36395    2022-09-01          2021              9          2022   
1            93162    2021-07-01          2020              4          2021   
2            98663    2021-04-01          2020              7          2021   
3           112635    2020-07-01          2020              4          2020   
4            29496    2023-05-01          2022              2          2023   

   Viewing_Month_Num  
0                  9  
1                  7  
2               

CONVERT DATA TYPES

In [36]:
# Convert numeric
df['Viewer_Rate'] = pd.to_numeric(df['Viewer_Rate'], errors='coerce')
df['Number_of_Views'] = pd.to_numeric(df['Number_of_Views'], errors='coerce')

In [37]:
# 2. Convert categorical first + fix missing
cat_cols = ['Film_Name', 'Category', 'Language']
for col in cat_cols:
    df[col] = df[col].astype("category")
    df[col] = df[col].cat.add_categories(["Unknown"])
    df[col] = df[col].fillna("Unknown")


In [38]:
# 4. Convert dates
df['Viewing_Month'] = pd.to_datetime(df['Viewing_Month'], format='%Y-%m-%d')
df['view_year'] = df['Viewing_Month'].dt.year
df['view_month'] = df['Viewing_Month'].dt.month

In [39]:
# Extract year and month for filtering
df['view_year'] = df['Viewing_Month'].dt.year
df['view_month'] = df['Viewing_Month'].dt.month

print("\nAFTER TYPE CONVERSION:")
print(df.info())


AFTER TYPE CONVERSION:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 460 entries, 0 to 459
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Film_Name          460 non-null    category      
 1   Release_Date       460 non-null    object        
 2   Category           460 non-null    category      
 3   Language           460 non-null    category      
 4   Viewer_Rate        460 non-null    float64       
 5   Number_of_Views    460 non-null    int64         
 6   Viewing_Month      460 non-null    datetime64[ns]
 7   Release_Year       460 non-null    int64         
 8   Release_Month      460 non-null    int64         
 9   Viewing_Year       460 non-null    int64         
 10  Viewing_Month_Num  460 non-null    int64         
 11  view_year          460 non-null    int32         
 12  view_month         460 non-null    int32         
dtypes: category(3), datetime64[ns](1), float6


 CLEAN DATA


In [40]:
# Remove duplicates
df = df.drop_duplicates()

In [41]:
# Fill missing numeric values with median
df['Viewer_Rate'] = df['Viewer_Rate'].fillna(df['Viewer_Rate'].median())
df['Number_of_Views'] = df['Number_of_Views'].fillna(df['Number_of_Views'].median())

In [42]:
# Drop rows with missing date info
df = df.dropna(subset=['Viewing_Month'])

REMOVE 2026 DATA FIRST

In [43]:
df = df[df['view_year'] != 2026]

 FEATURE ENGINEERING

In [44]:
# 1. Monthly Views
monthly_views = (
    df.groupby(['Film_Name', 'view_year', 'view_month'])['Number_of_Views']
    .sum()
    .reset_index()
    .rename(columns={'Number_of_Views': 'Monthly_Views'})
)

C:\Users\HP\AppData\Local\Temp\ipykernel_22580\2335214191.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['Film_Name', 'view_year', 'view_month'])['Number_of_Views']


In [45]:
# 2. Total Views per Film
total_views = (
    df.groupby('Film_Name')['Number_of_Views']
    .sum()
    .reset_index()
    .rename(columns={'Number_of_Views': 'Total_Views'})
)

C:\Users\HP\AppData\Local\Temp\ipykernel_22580\1879798740.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Film_Name')['Number_of_Views']


In [46]:
# 3. Average rating per Category
avg_rating_category = (
    df.groupby('Category')['Viewer_Rate']
    .mean()
    .reset_index()
    .rename(columns={'Viewer_Rate': 'Avg_Rating_Category'})
)

C:\Users\HP\AppData\Local\Temp\ipykernel_22580\2781774635.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Category')['Viewer_Rate']


In [47]:
# 4. Average rating per Language
avg_rating_lang = (
    df.groupby('Language')['Viewer_Rate']
    .mean()
    .reset_index()
    .rename(columns={'Viewer_Rate': 'Avg_Rating_Language'})
)

C:\Users\HP\AppData\Local\Temp\ipykernel_22580\2611079665.py:3: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby('Language')['Viewer_Rate']


MERGE FEATURES BACK TO MAIN DATA

In [48]:
df = df.merge(monthly_views,
              on=['Film_Name', 'view_year', 'view_month'],
              how='left')

df = df.merge(total_views, on='Film_Name', how='left')

df = df.merge(avg_rating_category, on='Category', how='left')
df = df.merge(avg_rating_lang, on='Language', how='left')


 FILTER DECEMBER 2025 

In [49]:
df_dec2025 = df[(df['view_year'] == 2025) & (df['view_month'] == 12)]

FINAL CLEANUP

In [50]:
df = df.drop(columns=['view_year', 'view_month'])

print("\nFINAL PROCESSED DATASET:")
print(df.head())



FINAL PROCESSED DATASET:
         Film_Name Release_Date     Category Language  Viewer_Rate  \
0  Chennai Express   2021-09-12      Romance    Hindi          4.5   
1   Mountain Trail   2020-04-27       Comedy  English          3.5   
2     Eternal Hope   2020-07-04      Romance  English          4.6   
3  Seoul Heartbeat   2020-04-17       Comedy  English          4.1   
4      Shadow Pact   2022-02-24  Documentary  English          4.3   

   Number_of_Views Viewing_Month  Release_Year  Release_Month  Viewing_Year  \
0            36395    2022-09-01          2021              9          2022   
1            93162    2021-07-01          2020              4          2021   
2            98663    2021-04-01          2020              7          2021   
3           112635    2020-07-01          2020              4          2020   
4            29496    2023-05-01          2022              2          2023   

   Viewing_Month_Num  Monthly_Views  Total_Views  Avg_Rating_Category  \
0    

SAVE OUTPUT FILE

In [51]:
df.to_csv("../../data/Processed_Film_Dataset.csv", index=False)
monthly_views.to_csv("../..//data/Feature_Monthly_Views.csv", index=False)
total_views.to_csv("../../data/Feature_Total_Views.csv", index=False)

print("\n✔ Files successfully generated:")
print("Processed_Film_Dataset.csv")
print("Feature_Monthly_Views.csv")
print("Feature_Total_Views.csv")


✔ Files successfully generated:
Processed_Film_Dataset.csv
Feature_Monthly_Views.csv
Feature_Total_Views.csv
